In [5]:
# Basic imports - NEED TO RUN IMPORT WARP
import sys
del sys.argv[1:] #  Necessry to run 'from warp import *' in IPython notebook without conflict.

#--- arguments passing to warp deck (these lines go before importing warp)
import warpoptions
warpoptions.parser.add_argument('--yoff',  dest='yoff',  type=float,default=0)
warpoptions.parser.add_argument('--tilt',  dest='tilt',  type=float,default=0)
warpoptions.parser.add_argument('--qbunch',dest='qbunch',type=float,default=2e-9)
warpoptions.parser.add_argument('--fname', dest='filename',type=str,default='diagGaussian')
warpoptions.parser.add_argument('--zscale', dest='zscale',type=float,default=1.0)
# Import warp-specific packages
from warp.init_tools import *
#import BDplotter as BD  #BD toolbox on NICADD cluster
import numpy as np 
import time 
import scipy.constants as cst

# -----------------------------------------------------------------------------
# Parameters (Modify the values below to suit your needs)
# -----------------------------------------------------------------------------

# General parameters
# ------------------
# Dimension of simulation ("3d", "circ", "2d", "1d")
dim = "3d"
# Total number of timesteps in the simulation (doesn't seem to do anything)
N_steps = 10
# Whether to run the simulation interactively (0:off, 1:on)
interactive = 1

# remove top slab
removeTop   = False

# deal with arguments
yoff=warpoptions.options.yoff
tilt=warpoptions.options.tilt
qbunch=warpoptions.options.qbunch
filename=warpoptions.options.filename
zscale=warpoptions.options.zscale

z0Beam = 3.5e-2   # starting location for the beam

# geometric parameter:
# --------------------
# size of DWA
L   =  4e-2    # along z (window length)
Ls  =  150e-3     # length of the structure (lz)
W   =  12.7e-3     # along x
a   =  2.5e-3/2.    # half vacuum gap
b   =  a+0.15e-3    # half conductor gap 
c   =  b+0e-3    # vacuum between dielectric and conducting boundary
Li  =  0.0       # padding (before and after structure)
eps =  3.75

# Simulation box
# --------------
# Number of grid cells in the longitudinal direction
Nz = 128 # *2
# Number of grid cells in transverse direction (represents Nr in "circ")
Nx = 128 # *2
# Number of grid cells in the 3rd dimension (not used for "2d" and "circ")
Ny = 64 # *2
# Dimension of the box in longitudinal direction (meters)
zmin = z0Beam-L/2.
zmax = z0Beam+L/2.
# Dimension of the box in transverse direction (box ranges from -xmax to xmax)
xmax = W/2.
# Dimension of the box in 3rd direction (not used for "2d" and "circ")
ymax = c

# Field boundary conditions (longitudinal and transverse respectively)
if removeTop: 
   f_boundxy = dirichlet
else:
   f_boundxy = dirichlet
f_boundz  = openbc

# particle boundaries
p_boundz  = absorb
p_boundxy = absorb

# Moving window (0:off, 1:on)
use_moving_window = 1
# Speed of the moving window (ignored if use_moving_window = 0)
v_moving_window = clight

# Diagnostics
# -----------
# Period of diagnostics (in number of timesteps)
diag_period = 20
# Whether to write the fields
write_fields = 1
# Whether to write the particles
write_particles = 1
# Whether to write the diagnostics in parallel
parallel_output = False

# Numerical parameters
# --------------------
# Field solver (0:Yee, 1:Karkkainen on EF,B, 3:Lehe)
stencil = 0
# Particle shape (1:linear, 2:quadratic, 3:cubic)
depos_order = 1
# Gathering mode (1:from cell centers, 4:from Yee mesh)
efetch = 1
# Particle pusher (0:Boris, 1:Vay)
particle_pusher = 1

# Current smoothing parameters
# ----------------------------
# Turn current smoothing on or off (0:off; 1:on)
use_smooth = 1
# Number of passes of smoother and compensator in each direction (x, y, z)
npass_smooth = array([[ 0 , 0 ], [ 0 , 0 ], [ 1 , 1 ]])
# Smoothing coefficients in each direction (x, y, z)
alpha_smooth = array([[ 0.5, 3.], [ 0.5, 3.], [0.5, 3./2]])
# Stride in each direction (x, y, z)
stride_smooth = array([[ 1 , 1 ], [ 1 , 1 ], [ 1 , 1 ]])


# input distribution
#-------------------
#filename   = 'setquad022020.DUTU250k'
# FB01.DUTent'
#PHSP       = BD.LoadElegantPhaseSpace(filename)
#beam_Np    = len(PHSP)
beam_uz   = 40/0.511
# Relative energy spread
beam_rel_energy_spread = 0.00
beam_Q     = 0.5e-9   #qbunch # bunch charge
# Number of macroparticles
beam_Np    = 250000
x0         = 0.0  # initial beam position 
y0         = yoff  # initial beam position 
z0         = z0Beam # initial beam position 
beam_weight= beam_Q / (beam_Np*echarge)


# Beam normalized emittance
beam_emit_x = 248e-6  # in m.rad
beam_emit_y = 3.7e-6  # in m.rad
# beam size at the waist will backpopagates
beam_xrms   = 1.866e-3
beam_yrms   = 0.130e-3
beam_zrms   = 640e-6

# -----------------------------------------------------------------------------
# Initialization of the simulation (Normal users should not modify this part.)
# -----------------------------------------------------------------------------

# Set some general options for warp
set_diagnostics( interactive )
set_boundary_conditions( f_boundz, f_boundxy, p_boundz, p_boundxy )
set_simulation_box( Nz, Nx, Ny, zmin, zmax, xmax, ymax, dim )
set_moving_window( use_moving_window, v_moving_window )

# See smoothing.py
set_smoothing_parameters( use_smooth, dim, npass_smooth,
                         alpha_smooth, stride_smooth )


# Set the numerical parameters only now: they affect the newly created species
top.ssnpid = nextpid()
set_numerics( depos_order, efetch, particle_pusher, dim)


# Setup the field solver object
# -----------------------------
em = initialize_em_solver( stencil, dim,
    npass_smooth, alpha_smooth, stride_smooth)
registersolver(em)

# Setup conductors & dielectrics
# ------------------------------
diel_slab_upper = Box(zsize=L,
                     ysize=b-a,
                     xsize= W,
                     xcent=0.5*(w3d.xmmin+w3d.xmmax),
                     ycent=0.5*(b+a),
                     zcent=0.5*(w3d.zmmin+w3d.zmmax),
                     permittivity=eps,condid=1)

diel_slab_lower = Box(zsize=L,
                     ysize=b-a,
                     xsize= W,
                     xcent=0.5*(w3d.xmmin+w3d.xmmax),
                     ycent=-0.5*(b+a),
                     zcent=0.5*(w3d.zmmin+w3d.zmmax),
                     permittivity=eps,condid=1)


if removeTop: 
   conductors=[diel_slab_lower]
else:
   conductors=[diel_slab_upper, diel_slab_lower]


for cond in conductors:
  em.installconductor(cond, dfill = largepos) # Install conductors with EM solver

usenewconductorgeneration()
#scraper = ParticleScraper(conductors = conductors)

# beam setup
# -------------------------

# Creation of the species
# -----------------------


bgam      = beam_uz
vth_xrms  = beam_emit_x/beam_xrms*clight/bgam
vth_yrms  = beam_emit_y/beam_yrms*clight/bgam
vth_zrms  = beam_rel_energy_spread*beam_uz*clight

beam_weight = beam_Q / (beam_Np*echarge)
beam = Species(type=Electron, weight=beam_weight, name='beam')
beam.add_gaussian_dist(beam_Np, beam_xrms, beam_yrms, beam_zrms,
			vthx=vth_xrms, vthy=vth_yrms, vthz=vth_zrms,
			xmean=0, ymean=0, zmean=z0Beam,
			vxmean=0, vymean=0, vzmean=beam_uz*clight,
			vxdiv =0, vydiv =0, vzdiv=0,
			lmomentum=True, w=1.)


# add correlated divergence (backpropagate up to the structrure entrance):


print ('>>> x emittance [um] =', 1e6*np.sqrt(np.std(beam.xp)**2*np.std(beam.uxp)**2)/clight*bgam)
print ('>>> y emittance [um] =', 1e6*np.sqrt(np.std(beam.yp)**2*np.std(beam.uyp)**2)/clight*bgam)

beam.xp = beam.xp - Ls/2*beam.uxp/clight
beam.yp = beam.yp - Ls/2*beam.uyp/clight

#beam.gaminv[np.abs(beam.xp) >= 2.5*beam_xrms ] = 0.
#beam.gaminv[np.abs(beam.yp) >= 2.5*beam_yrms ] = 0.
#beam.gaminv[np.abs(beam.zp-z0Beam) >= 3*beam_zrms ] = 0.

# Set the numerical parameters only now: they affect the newly created species
top.ssnpid = nextpid()
set_numerics( depos_order, efetch, particle_pusher, dim)


# impart a beam tilt in X-Y 
theta=tilt
tmpx = beam.xp*np.cos(theta*np.pi/180) + beam.yp*np.sin(theta*np.pi/180)
tmpy =-beam.xp*np.sin(theta*np.pi/180) + beam.yp*np.cos(theta*np.pi/180)

# nominally thetap=theta
thetap=tilt
tmppx = beam.uxp*np.cos(theta*np.pi/180) + beam.uyp*np.sin(theta*np.pi/180)
tmppy =-beam.uxp*np.sin(theta*np.pi/180) + beam.uyp*np.cos(theta*np.pi/180)


beam.xp  = tmpx-np.mean(tmpx)+x0
beam.yp  = tmpy-np.mean(tmpy)+y0
beam.uxp = tmppx
beam.uyp = tmppy 



print ("gamma=", np.mean(1/beam.getgaminv()))
print ("pz=", np.mean(beam.getuz()))
print ("E=", np.mean(beam.getke()))

initialize_beam_fields( em, dim, beam, w3d, top )

# Setup the diagnostics
# ---------------------
timestr = time.strftime("%Y%m%d_%H%M%S")
diag_dir=filename+'_yoffum_'+str(yoff*1e6).zfill(4)+'_tiltdeg_'+ \
          str(tilt).zfill(4)+'_'+str(timestr)
remove_existing_directory( [diag_dir] )
if write_fields == 1:
    diag1 = FieldDiagnostic( period=diag_period, top=top, w3d=w3d, em=em,
                comm_world=comm_world, lparallel_output=parallel_output,
                write_dir=diag_dir )
    installafterstep( diag1.write )
if write_particles == 1:
    diag2 = ParticleDiagnostic( period=diag_period, top=top, w3d=w3d,
            species={species.name : species for species in listofallspecies},
            particle_data=["position","momentum","weighting","id"],
            comm_world=comm_world, lparallel_output=parallel_output,
            write_dir=diag_dir  )
    installafterstep( diag2.write )

def scrapebeam():

  if removeTop:
      beam.gaminv[beam.yp <= -a ] = 0.
  else:
      beam.gaminv[np.abs(beam.yp) >= a ] = 0.

installparticlescraper(scrapebeam)


def myplots():
#    if top.it%5<>0:return
    
    fma()
    window(0)

    #EM.pfex(direction = 1, l_transpose = 1, view = 1, cmin = -E0, cmax = E0)
    em.pfez(direction = 0, l_transpose = 1)

    # Plot conductors in the X-Z plane
    pfzy(cond = 0, plotsg = 1, fill = 0, plotphi = 0, plotselfe = 0)
    
    # Plot particle projections
    beam.ppzy() # See warpplots.py, function ppgeneric for argument defintions

    # Define graph dimensions
    limits(w3d.zmmin+top.zgrid, w3d.zmmax+top.zgrid, w3d.ymmin, w3d.ymmax)
    #limits(w3d.zmmin, w3d.zmmax, 4.0e-6, w3d.xmmax)

    #ex=EM.gatherex()
    #x=w3d.xmmin+arange(w3d.nx+1)*w3d.dx
    #ex=ex[:,w3d.ny/2,w3d.nz/2]
#    ez=em.gatherez()
#    z=w3d.zmmin+arange(w3d.nz+1)*w3d.dz
#    ez=ez[int(w3d.nx/2),int(w3d.ny/2),:]
#    plsys(10) # Selects lower plot
#    pla(ez, z, width = 4)

    refresh()
if interactive == 1:
    installafterstep(myplots)

print('\n Initialization complete \n')


# -----------------------------------------------------------------------------
# Simulation loop 
# -----------------------------------------------------------------------------

# Non-interactive mode
if interactive==0:
    n_stepped=0
    while w3d.zmmax+top.zgrid < Ls+z0:
        step(10)
        n_stepped = n_stepped + 20

    dump()
    printtimers()

# Interactive mode
elif interactive==1:
    n_stepped=0
    while w3d.zmmax+top.zgrid < Ls+z0:
        step(10)
        n_stepped = n_stepped + 20

    dump()
    printtimers()


ArgumentError: argument --yoff: conflicting option string: --yoff